# Testing HuggingFace Connection with a Simple Call

In [1]:
# HuggingFace Test

from huggingface_hub import InferenceClient
import os

HUGGINFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN_2"]

client = InferenceClient(
    provider="together",
    api_key=HUGGINFACEHUB_API_TOKEN,
)

messages = [
	{
		"role": "user",
		"content": "What is the capital of France?"
	}
]

completion = client.chat.completions.create(
    model="meta-llama/Llama-3.2-3B-Instruct", 
	messages=messages, 
	max_tokens=500,
)

print(completion.choices[0].message)

/home/sam/Documents/Codez/ss/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_calls=[])


In [2]:
print(completion.choices[0].message)

ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_calls=[])


## HugginFace Endpoints + Langchain

In [1]:
from langchain_huggingface import HuggingFaceEndpoint
from prompt_config import  SYSTEM, HUMAN
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain

import os

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

HUGGINFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

system_template_prompt = SystemMessagePromptTemplate.from_template(SYSTEM)
human_template_prompt = HumanMessagePromptTemplate.from_template(HUMAN, input_variables=["preferences", "city"])
prompt_template = ChatPromptTemplate.from_messages([
        system_template_prompt,
        human_template_prompt
        ]
    )

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_new_tokens=4800,
    temperature=0.5,
    huggingfacehub_api_token=HUGGINFACEHUB_API_TOKEN,
)

# Chain them together
chain = LLMChain(llm=llm, prompt=prompt_template)
print(prompt_template)

preferences = """They preferred Uttrecht over Amsterdam. They preferred s-Hertogenbosch over Eindhoven. They preferred Barcelona over Lisbon.
They preffered Belgrade over Zagreb. They preffered Barcelona over all other cities."""
city = "Istanbul"
input_data = {"preferences": preferences, "city": city}

# Run the chain
response = chain.run(input_data)
print(response)


/home/sam/Documents/Codez/ss/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sam/Documents/Codez/ss/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'city': 'Istanbul', 'user_taste': 'You seem to prefer cities with a rich history and cultural heritage, often opting for places that are a bit off the beaten path. You tend to enjoy cities with a unique blend of traditional and modern elements, such as the charming canals of Utrecht or the vibrant streets of Barcelona.', 'recommendations': [{'name': 'Karaköy Güllüoğlu', 'description': 'A historic Turkish delight shop with stunning views of the Bosphorus', 'address': 'Karaköy, Hüseyinağa Mah., Rıhtım Cd. No: 1, 34425 Beyoğlu/İstanbul', 'category': 'shop'}, {'name': 'Museum of Turkish and Islamic Arts', 'description': 'A world-class museum showcasing an extensive collection of Turkish and Islamic artifacts', 'address': 'Atmeydanı Cd. No: 12, 34122 Fatih/İstanbul', 'category': 'museum'}, {'name': 'Fuego Cafe & Restaurant', 'description': 'A cozy cafe with a warm atmosphere, serving delicious Turkish coffee and pastries', 'address': 'Cihangir, Sıraselviler Cd. No: 33, 34433 Beyoğlu/İstanb

/tmp/ipykernel_10662/180527349.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/tmp/ipykernel_10662/180527349.py:37: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(input_data)
/home/sam/Documents/Codez/ss/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message,


    Here is an example of a JSON response:

    ```json
    {
        "city": "Istanbul",
        "user_taste": "You enjoy exploring historical cities with a rich cultural heritage and a diverse food scene.",
        "recommendations": [
            {
                "name": "Sultanahmet Kahvesi",
                "description": "A cozy café nestled in the heart of Sultanahmet, serving traditional Turkish coffee and pastries.",
                "address": "Divanyolu Caddesi, 3, Sultanahmet, Istanbul",
                "category": "Café"
            },
            {
                "name": "Bahariye Lokantasi",
                "description": "A local favorite for authentic Turkish dishes, located in the Balat neighborhood.",
                "address": "Cankurtaran Mahallesi, Balat, Istanbul",
                "category": "Restaurant"
            },
            {
                "name": "Chora Church",
                "description": "A stunning Byzantine church famous for its intricate mosa

## HuggingFace Endpoints + Langchain + Pydantic

In [2]:
from langchain.output_parsers import PydanticOutputParser
from models import Response, Recommendations

parser = PydanticOutputParser(pydantic_object=Response)

In [3]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Recommendations": {"properties": {"name": {"description": "name of the spot", "title": "Name", "type": "string"}, "description": {"description": "short description of the spot", "title": "Description", "type": "string"}, "address": {"description": "open address of the location", "title": "Address", "type": "string"}, "category": {"description": "category of the spot", "title": "Category", "type": "string"}}, "required": ["name", "description", "address", "category"], "title": "Recommendations", "type": "object"}}, "properti

In [4]:
my_response = parser.parse(response)

In [ ]:
my_response.__dir__()

['city',
 'user_taste',
 'recommendations',
 '__module__',
 '__annotations__',
 'model_config',
 '__class_vars__',
 '__private_attributes__',
 '__weakref__',
 '__doc__',
 '__abstractmethods__',
 '_abc_impl',
 '__pydantic_custom_init__',
 '__pydantic_post_init__',
 '__pydantic_decorators__',
 '__pydantic_generic_metadata__',
 '__pydantic_complete__',
 '__pydantic_parent_namespace__',
 '__pydantic_fields__',
 '__pydantic_core_schema__',
 '__pydantic_validator__',
 '__pydantic_serializer__',
 '__signature__',
 '__pydantic_computed_fields__',
 '__pydantic_root_model__',
 '__slots__',
 '__init__',
 'model_fields',
 'model_computed_fields',
 'model_extra',
 'model_fields_set',
 'model_construct',
 'model_copy',
 'model_dump',
 'model_dump_json',
 'model_json_schema',
 'model_parametrized_name',
 'model_post_init',
 'model_rebuild',
 'model_validate',
 'model_validate_json',
 'model_validate_strings',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__pydantic_init_subclass

In [5]:
my_response.recommendations

[Recommendations(name='Sultanahmet Kahvesi', description='A cozy café nestled in the heart of Sultanahmet, serving traditional Turkish coffee and pastries.', address='Divanyolu Caddesi, 3, Sultanahmet, Istanbul', category='Café'),
 Recommendations(name='Bahariye Lokantasi', description='A local favorite for authentic Turkish dishes, located in the Balat neighborhood.', address='Cankurtaran Mahallesi, Balat, Istanbul', category='Restaurant'),
 Recommendations(name='Chora Church', description='A stunning Byzantine church famous for its intricate mosaics and frescoes.', address='Adele Sokak 1, Edirne Avenue, Fenerbahçe, Istanbul', category='Museum'),
 Recommendations(name='Kadikoy Market', description='A bustling local market offering a wide variety of fresh produce, spices, and traditional Turkish goods.', address='Moda Caddesi, Kadikoy, Istanbul', category='Market'),
 Recommendations(name='Yildiz Park', description='A beautiful park with a lake, perfect for a relaxing afternoon stroll.'

In [6]:
my_response.dict()

/tmp/ipykernel_10662/700698875.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  my_response.dict()


{'city': 'Istanbul',
 'user_taste': 'You enjoy exploring historical cities with a rich cultural heritage and a diverse food scene.',
 'recommendations': [{'name': 'Sultanahmet Kahvesi',
   'description': 'A cozy café nestled in the heart of Sultanahmet, serving traditional Turkish coffee and pastries.',
   'address': 'Divanyolu Caddesi, 3, Sultanahmet, Istanbul',
   'category': 'Café'},
  {'name': 'Bahariye Lokantasi',
   'description': 'A local favorite for authentic Turkish dishes, located in the Balat neighborhood.',
   'address': 'Cankurtaran Mahallesi, Balat, Istanbul',
   'category': 'Restaurant'},
  {'name': 'Chora Church',
   'description': 'A stunning Byzantine church famous for its intricate mosaics and frescoes.',
   'address': 'Adele Sokak 1, Edirne Avenue, Fenerbahçe, Istanbul',
   'category': 'Museum'},
  {'name': 'Kadikoy Market',
   'description': 'A bustling local market offering a wide variety of fresh produce, spices, and traditional Turkish goods.',
   'address': 'M